In [29]:
import asyncio
import random
import time

from contextlib import contextmanager

@contextmanager
def timed(name=''):
    t0 = time.perf_counter()
    yield
    t1 = time.perf_counter()
    print(f"Executed {name} in {t1 - t0}s")

async def foo(i):
    slept = random.randint(0, 5)
#     print(f"sleeping {slept}s")
    await asyncio.sleep(slept)
#     print(f"slept {slept}s")
    return i


async def async_range(*args, **kwargs):
    for x in range(*args, **kwargs):
        yield x

    
async def slow_bar(n):
    """
    Seems kinda silly, but async for does _not_ run in parallel.
    
    It just allows iterating over an async generator.
    """
    values = []
    async for i in async_range(n):  # runs serially
        value = await foo(i)
        print(value)
        values.append(value)
    return values

with timed("slow_bar"):
    await slow_bar(3)

async def fast_bar(n):
    values = []
    for value in asyncio.as_completed([foo(i) for i in range(n)]):
        result = await value
        values.append(result)
#         print(result)
    return values


with timed("fast_bar"):
    await fast_bar(100)

0
1
2
Executed slow_bar in 12.01275944200006s
Executed fast_bar in 5.010510840000279s


In [30]:
!pip install aiofiles

You should consider upgrading via the '/Users/lwiman/personal/stuff/python/notebooks/.venv/bin/python -m pip install --upgrade pip' command.


In [34]:
import pathlib
import aiofiles
pathlib.Path('/tmp/foo').write_text('bar')
async with aiofiles.open('/tmp/foo', mode='r') as f:
    contents = await f.read()
print(contents)

bar
